In [1]:
%load_ext autoreload
%autoreload 2
import os
import sys

project_module_path = os.path.abspath(os.path.join('../'))
if project_module_path not in sys.path:
    print(f'Adding {project_module_path} to system paths')
    sys.path.append(project_module_path)

# %env AWS_PROFILE=development

# Base Model v1

## Target
- Basic setup: Loading data, train and test loops
- Basic model with regularization to prevent overfitting and improve generalization,
    - Batch-normalisation to reduce the risk of memorizing noise in the training data
    - Dropout to perform better on new and unseen data.
- Global Average Pooling instead of a fully connected layer at the end
## Result
- Parameters: 24,986
- Accuracy:
| Accuracy | Best | Final |
| --- | --- | --- |
| Train | 99.77 | 99.77
| Test | 99.81 | 99.40
## Analysis
- Model is moderately heavy
- Overfitting (train acc > test acc)


## Import all the libraries

In [2]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from tqdm import tqdm
from torchvision import datasets, transforms
from torch.utils.tensorboard import SummaryWriter

# !pip install albumentations==2.0.8
# !pip install torchsummary
from torchsummary import summary

In [3]:
import logging

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

logging.info("logging configured")  # calling the root logger

INFO:root:logging configured


## Configure device

In [5]:
use_cuda = torch.cuda.is_available()
use_mps = torch.backends.mps.is_available()

device: torch.device = None
if use_cuda:
    device = torch.device("cuda")
elif use_mps:
    device = torch.device("mps")
else:
    device = torch.device("cpu")

assert device is not None, "device must not be None"
assert isinstance(device, torch.device), "device must be an instance of torch.device"

use_cuda

True

## Download the dataset and configure loaders

In [6]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

from data import train_transforms, test_transforms

torch.manual_seed(1)
BATCH_SIZE = 128

# Transforms
# train_transforms = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Normalize((0.1307,), (0.3081,))
# ])
# test_transforms = transforms.Compose([
#     transforms.ToTensor(),
#     transforms.Normalize((0.1307,), (0.3081,))
# ])

train = datasets.CIFAR10('../../data', train=True, download=True, transform=train_transforms)
test = datasets.CIFAR10('../../data', train=False, download=True, transform=test_transforms)

dataloader_args = dict(shuffle=True, batch_size=BATCH_SIZE, num_workers=2, pin_memory=True) if (use_cuda or use_mps) else dict(shuffle=True, batch_size=64)

# train dataloader
train_loader = torch.utils.data.DataLoader(train, **dataloader_args)

# test dataloader
test_loader = torch.utils.data.DataLoader(test, **dataloader_args)


100%|██████████| 170M/170M [00:13<00:00, 13.0MB/s] 


## Model summary

In [7]:
from models import ModelV1
summary(ModelV1(), input_size=(3, 28, 28))

RuntimeError: Input type (torch.cuda.FloatTensor) and weight type (torch.FloatTensor) should be the same

## Train and evaluate the model

### Configuration

In [9]:
from pytorch_play import Config, Trainer

summaryWriter=SummaryWriter("../../logs/cifar10")
tConfig = Config(
    summaryWriter=summaryWriter,
    enable_lr_scheduler=True,
    epochs=100
)
print(f"tConfig: {tConfig}")

modelV1 = ModelV1().to(device)

tConfig: batch_size=128 epochs=100 lr=0.01 enable_lr_scheduler=True momentum=0.9 seed=1 log_dir='logs/mnist' summaryWriter=<torch.utils.tensorboard.writer.SummaryWriter object at 0x7b203232e810>


### Start the loop

In [10]:
import warnings
import os

warnings.filterwarnings('ignore')
os.environ["ALBUMENTATIONS_DISABLE_VERSION_CHECK"] = "1"
os.environ["NO_ALBUMENTATIONS_UPDATE"] = "1"

trainer = Trainer(tConfig)
trainer.fit(modelV1, device, train_loader, test_loader, modelV1.name())

Epoch 1:   0%|          | 0/391 [00:00<?, ?it/s]

Epoch 1: 100%|██████████| 391/391 [00:24<00:00, 15.69it/s, batch_id=390, loss=1.5951]
INFO:pytorch_play.trainer:[TRAIN ModelV1] Epoch 01 - Accuracy: 17015/50000 (34.03%)
INFO:pytorch_play.trainer:[TEST ModelV1] Epoch 01 - Loss: 1.5062, Accuracy: 4577/10000 (45.77%)
Epoch 2: 100%|██████████| 391/391 [00:23<00:00, 16.45it/s, batch_id=390, loss=1.5107]
INFO:pytorch_play.trainer:[TRAIN ModelV1] Epoch 02 - Accuracy: 22788/50000 (45.58%)
INFO:pytorch_play.trainer:[TEST ModelV1] Epoch 02 - Loss: 1.2721, Accuracy: 5527/10000 (55.27%)
Epoch 3: 100%|██████████| 391/391 [00:23<00:00, 16.41it/s, batch_id=390, loss=1.3191]
INFO:pytorch_play.trainer:[TRAIN ModelV1] Epoch 03 - Accuracy: 25589/50000 (51.18%)
INFO:pytorch_play.trainer:[TEST ModelV1] Epoch 03 - Loss: 1.2201, Accuracy: 5633/10000 (56.33%)
Epoch 4: 100%|██████████| 391/391 [00:23<00:00, 16.39it/s, batch_id=390, loss=1.2822]
INFO:pytorch_play.trainer:[TRAIN ModelV1] Epoch 04 - Accuracy: 27398/50000 (54.80%)
INFO:pytorch_play.trainer:[TEST 